In [0]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import tensorflow
import numpy as np
import argparse
import cv2
import os


face="simple_face_detector"
model1="mask_detector.model"
confidence1=0.2  # For face detection


In [0]:
!unzip mask_detector.zip

Archive:  mask_detector.zip
  inflating: mask_detector.model     
  inflating: simple_face_detector/deploy.prototxt  
  inflating: simple_face_detector/res10_300x300_ssd_iter_140000.caffemodel  


In [0]:
from google.colab import files
content_path = files.upload()             
image1 = list(content_path)[0]
print(image1)

Saving face-masks-Article-202005191632.jpg to face-masks-Article-202005191632.jpg
face-masks-Article-202005191632.jpg


In [0]:
def load():
	proto = os.path.sep.join([face, "deploy.prototxt"])
	weights = os.path.sep.join([face, "res10_300x300_ssd_iter_140000.caffemodel"])
	net = cv2.dnn.readNet(proto, weights)

	model = load_model(model1)
	return model, net

def process(model, net):
	image = cv2.imread(image1)
	origne = image.copy()
	(h, w) = image.shape[:2]

	blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
		(104.0, 177.0, 123.0))
	net.setInput(blob)
	detections = net.forward()
  
	for i in range(0, detections.shape[2]):
		confidence = detections[0, 0, i, 2]
		if confidence > confidence1:
			box = detections[0, 0, i, 3:7] * np.array([w,h,w,h])
			(startX, startY, endX, endY) = box.astype("int")

			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w-1, endX), min(h-1, endY))


			# extract face ROI, convert it to RGB channel
			face = image[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224,224))
			face = img_to_array(face)
			face = preprocess_input(face)
			face = np.expand_dims(face, axis=0)

			#pass the face through the model to determince if the face has a mask or not
			(mask, withoutMask) = model.predict(face)[0]

			label = "Mask" if mask > withoutMask else "No Mask"
			color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

			label = "{}: {:.2f}%".format(label, max(mask, withoutMask)*100)

			cv2.putText(image, label, (startX, startY-10),
				cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
			cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
	cv2.imwrite("result.png",image) #cv2.imshow("Ouput", image)
	return(detections)

	#if cv2.waitKey(0) == ord('Q'):
	#	cv2.destroyAllWindows()


	

In [0]:
gpus = tensorflow.config.experimental.list_physical_devices('GPU')
print("------------"+str(len(tensorflow.config.experimental.list_physical_devices('CPU'))))
if len(gpus) == 0:
  gpus = tensorflow.config.experimental.list_physical_devices('CPU')
try:
  tensorflow.config.experimental.set_virtual_device_configuration(gpus[0],
  [tensorflow.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])

  print(" - [x]: Load the model from disk... ->")
  model, net = load()

  print(" - [x]: Trying to predict ... ->")
  detections = process(model,  net)
except Exception as e:
  raise e

------------1
 - [x]: Load the model from disk... ->
 - [x]: Trying to predict ... ->
[[[[0.         1.         0.9189106  ... 0.59847987 0.9940617
    0.7182056 ]
   [0.         1.         0.910735   ... 0.4879036  0.20386873
    0.6143639 ]
   [0.         1.         0.89528954 ... 0.57715064 0.57544065
    0.69282776]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]


In [0]:
# Print confidence of the face detected:

for i in range(0, detections.shape[2]):
  confidence = detections[0, 0, i, 2]
  if confidence > confidence1:
    print(confidence)

0.9189106
0.910735
0.89528954
0.88994944
0.8516732
0.8448359
0.8094372
0.6862237
0.5665183
0.39056483
0.25843647
0.22367479
0.21122274
